<a href="https://colab.research.google.com/github/DrRiteshAjoodha/CV-of-Dr.-Ritesh-Ajoodha/blob/master/Comorbidity_influence_between_disease_processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === One-cell Colab script with safe setup + auto-restart ===
import sys, os, subprocess, json, datetime

def need_setup():
    try:
        import numpy as np
        import sklearn
        return not (int(np.__version__.split('.')[0]) >= 2 and int(sklearn.__version__.split('.')[0]) >= 1 and int(sklearn.__version__.split('.')[1]) >= 6)
    except Exception:
        return True

if need_setup():
    print("🔧 Setting up compatible versions (NumPy ≥2, scikit-learn ≥1.6, pgmpy) ...")
    # Pin to versions that play nicely in current Colab images
    cmds = [
        "pip", "install", "-q", "--upgrade",
        "numpy>=2.0.0",
        "pandas>=2.2.2",
        "scikit-learn>=1.6.0",
        "networkx>=3.2.1",
        "matplotlib>=3.8.4",
        "pgmpy==0.1.25"
    ]
    subprocess.check_call([sys.executable, "-m"] + cmds)
    print("✅ Setup complete. The runtime will restart once to load new NumPy. Please re-run this cell afterwards.")
    # Force a runtime restart to avoid binary incompatibility
    os.kill(os.getpid(), 9)

# ----------------- Everything below runs AFTER restart -----------------
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score

# --------- Synthetic data generator ---------
def _clip_state(x, k):
    return int(max(0, min(k - 1, x)))

def generate_synthetic_comorbidity(n_patients=300, timesteps=20, random_state=42, K=3):
    """
    3 diseases: diab, ckd, htn
    3 labs/vitals: hba1c, creat, bp
    True influences:
        diab_t -> ckd_{t+1}
        htn_t  -> ckd_{t+1}
        diab_t -> hba1c_{t+1}
        ckd_t  -> creat_{t+1}
    + self-transitions for all
    """
    rng = np.random.default_rng(random_state)
    rows = []
    for pid in range(n_patients):
        diab = rng.integers(0, K); ckd  = rng.integers(0, K); htn  = rng.integers(0, K)
        hba1c = rng.integers(0, K); creat = rng.integers(0, K); bp = rng.integers(0, K)
        for t in range(timesteps):
            rows.append(dict(patient_id=pid, t=t, diab=diab, ckd=ckd, htn=htn, hba1c=hba1c, creat=creat, bp=bp))
            diab_next = _clip_state(diab + rng.choice([-1,0,0,0,1]), K)
            htn_next  = _clip_state(htn  + rng.choice([-1,0,0,0,1]), K)
            ckd_shift = (1 if diab>0 else 0) + (1 if htn>0 else 0)
            ckd_next  = _clip_state(ckd + rng.choice([-1,0,0,1]) + (1 if ckd_shift>0 and rng.random()<0.6 else 0), K)
            hba1c_next = _clip_state(hba1c + (1 if diab>0 and rng.random()<0.7 else 0) + rng.choice([-1,0,0,1]), K)
            creat_next = _clip_state(creat + (1 if ckd>0  and rng.random()<0.6 else 0) + rng.choice([-1,0,0,1]), K)
            bp_next    = _clip_state(bp    + (1 if htn>0  and rng.random()<0.6 else 0) + rng.choice([-1,0,0,1]), K)
            diab, ckd, htn, hba1c, creat, bp = diab_next, ckd_next, htn_next, hba1c_next, creat_next, bp_next
    return pd.DataFrame(rows).sort_values(["patient_id","t"]).reset_index(drop=True)

# --------- Two-slice utilities ---------
VARS = ["diab","ckd","htn","bp","hba1c","creat"]

def build_two_slice(df: pd.DataFrame, vars: List[str]) -> pd.DataFrame:
    cur = df.copy(); cur["t1"] = cur["t"] + 1
    cur = cur.rename(columns={v: f"{v}_t" for v in vars})[["patient_id","t1"] + [f"{v}_t" for v in vars]]
    nxt = df.rename(columns={v: f"{v}_t1" for v in vars})[["patient_id","t"] + [f"{v}_t1" for v in vars]]
    merged = pd.merge(cur, nxt, left_on=["patient_id","t1"], right_on=["patient_id","t"], how="inner")
    return merged.drop(columns=["t","t1"]).reset_index(drop=True)

def self_edges(vars: List[str]):  return [(f"{v}_t", f"{v}_t1") for v in vars]
def cross_edges(vars: List[str]): return [(f"{u}_t", f"{v}_t1") for u in vars for v in vars if u != v]

# --------- Greedy BIC search ---------
def score_model(structure: List[Tuple[str,str]], data: pd.DataFrame) -> float:
    model = BayesianNetwork(structure)
    model.add_nodes_from(list(data.columns))
    model.fit(data, estimator=MaximumLikelihoodEstimator)
    return BicScore(data).score(model)

def greedy_add_edges(data: pd.DataFrame, all_nodes: List[str], whitelist, start_edges, max_parents=2, max_iters=50):
    current = list(start_edges)
    best_score = score_model(current, data)
    def parent_count(edges):
        cnt = {n:0 for n in all_nodes}
        for u,v in edges: cnt[v] += 1
        return cnt
    parents = parent_count(current)
    candidates = list(set(whitelist) - set(current))
    it = 0
    while it < max_iters:
        it += 1
        best_delta, best_edge = 0.0, None
        for e in candidates:
            u,v = e
            if parents[v] >= max_parents: continue
            trial = current + [e]
            try:
                s = score_model(trial, data)
            except Exception:
                continue
            delta = s - best_score
            if delta > best_delta:
                best_delta, best_edge = delta, e
        if not best_edge or best_delta <= 0: break
        current.append(best_edge); best_score += best_delta; parents[best_edge[1]] += 1; candidates.remove(best_edge)
    return {"edges": current, "bic": best_score, "iterations": it}

# --------- Fit + evaluate ---------
def fit_bn(structure, train_df):
    m = BayesianNetwork(structure)
    m.add_nodes_from(list(train_df.columns))
    m.fit(train_df, estimator=MaximumLikelihoodEstimator)
    return m

def evaluate_nextstep(model, test_df, vars):
    infer = VariableElimination(model)
    X_cols = [f"{v}_t" for v in vars]
    out = {}
    for v in vars:
        y_true = test_df[f"{v}_t1"].values
        preds = []
        for i in range(len(test_df)):
            evidence = {col: int(test_df.iloc[i][col]) for col in X_cols}
            q = infer.query(variables=[f"{v}_t1"], evidence=evidence, show_progress=False)
            preds.append(int(np.argmax(q.values)))
        out[v] = {"accuracy": float((np.array(preds) == y_true).mean())}
    return out

def split_train_test(two_slice_df, test_frac=0.25, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(len(two_slice_df)); rng.shuffle(idx)
    split = int(len(idx) * (1 - test_frac))
    return two_slice_df.iloc[idx[:split]].reset_index(drop=True), two_slice_df.iloc[idx[split:]].reset_index(drop=True)

# --------- Demo ---------
def run_demo(n_patients=300, timesteps=20, seed=42, max_parents=2, test_frac=0.25):
    df = generate_synthetic_comorbidity(n_patients=n_patients, timesteps=timesteps, random_state=seed)
    two = build_two_slice(df, VARS)
    train_df, test_df = split_train_test(two, test_frac=test_frac, seed=seed)

    self_only = self_edges(VARS)
    indep_bic = score_model(self_only, train_df)
    indep_model = fit_bn(self_only, train_df)
    indep_metrics = evaluate_nextstep(indep_model, test_df, VARS)

    wl = self_only + cross_edges(VARS)
    result = greedy_add_edges(train_df, list(train_df.columns), wl, self_only, max_parents=max_parents, max_iters=50)
    infl_edges, infl_bic = result["edges"], result["bic"]
    infl_model = fit_bn(infl_edges, train_df)
    infl_metrics = evaluate_nextstep(infl_model, test_df, VARS)

    return {
        "timestamp": datetime.datetime.utcnow().isoformat() + "Z",
        "config": {"n_patients": n_patients, "timesteps": timesteps, "seed": seed,
                   "max_parents": max_parents, "test_frac": test_frac},
        "independent": {"bic": indep_bic, "edges": self_only, "metrics": indep_metrics},
        "influence": {"bic": infl_bic, "edges": infl_edges, "metrics": infl_metrics}
    }

report = run_demo(n_patients=300, timesteps=20, seed=42, max_parents=2, test_frac=0.25)

print("\n=== Summary ===")
print("Independent BIC:", report["independent"]["bic"])
print("Influence   BIC:", report["influence"]["bic"])
print("\nEdges learned (influence model):")
for u,v in report["influence"]["edges"]:
    print(f"  {u} -> {v}")
print("\nPer-variable next-step accuracy (independent):")
for v, m in report["independent"]["metrics"].items(): print(f"  {v}: {m['accuracy']:.3f}")
print("Per-variable next-step accuracy (influence):")
for v, m in report["influence"]["metrics"].items(): print(f"  {v}: {m['accuracy']:.3f}")

with open("run_report.json","w") as f:
    json.dump(report, f, indent=2)
print("\nSaved: run_report.json")



=== Summary ===
Independent BIC: -63834.04695506026
Influence   BIC: -63559.41164167458

Edges learned (influence model):
  diab_t -> diab_t1
  ckd_t -> ckd_t1
  htn_t -> htn_t1
  bp_t -> bp_t1
  hba1c_t -> hba1c_t1
  creat_t -> creat_t1
  diab_t -> hba1c_t1
  htn_t -> bp_t1
  ckd_t -> creat_t1

Per-variable next-step accuracy (independent):
  diab: 0.734
  ckd: 0.779
  htn: 0.739
  bp: 0.754
  hba1c: 0.746
  creat: 0.795
Per-variable next-step accuracy (influence):
  diab: 0.734
  ckd: 0.779
  htn: 0.739
  bp: 0.761
  hba1c: 0.767
  creat: 0.799

Saved: run_report.json
